In [32]:
import os

In [33]:
%pwd

"C:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project"

In [34]:
os.chdir("C:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project")

In [35]:
%pwd

"C:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project"

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [37]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:

    # constructor initialised by config & params filepaths
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    #Read yaml files & return it's content

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    #Create dirctory artfacts_root => for "config.artifacts_root" here is the use of ConfigBox
        create_directories([self.config.artifacts_root])
        

    # return of this method is DataIngestionConfig    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
    # creating root_dir
        create_directories([config.root_dir])
    
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
        )

        return data_transformation_config   

In [38]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [39]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config= config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(example_batch):
        input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with tokenizer.as_target_tokenizer():
            target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self) :
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_examples_to_features,batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [40]:
try:
    config = ConfigurationManager()
    data_transformation_config= config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-11-17 00:46:46,727: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-17 00:46:46,735: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-17 00:46:46,740: INFO: common: created directory at: artifacts]
[2023-11-17 00:46:46,744: INFO: common: created directory at: artifacts/data_transformation]


NameError: name 'data_transformation_config' is not defined